In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn import *
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from IPython.display import display_markdown as mkdown # as print

def nl():
    print('\n')

for f in os.listdir("C:/Users/vishw/PycharmProjects/kaggle/Bimbo/New folder/"):
    print(f.ljust(30) + str(round(os.path.getsize("C:/Users/vishw/PycharmProjects/kaggle/Bimbo/New folder/" + f) / 1000000, 2)) + 'MB')

C:\Users\vishw\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


cliente_tabla.csv             21.25MB
producto_tabla.csv            0.11MB
sample_submission.csv         68.88MB
test.csv                      251.11MB
town_state.csv                0.03MB
train.csv                     3199.36MB


C:\Users\vishw\Anaconda3\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [2]:
df_train = pd.read_csv("C:/Users/vishw/PycharmProjects/kaggle/Bimbo/New folder/train.csv", nrows=500000)
df_test = pd.read_csv("C:/Users/vishw/PycharmProjects/kaggle/Bimbo/New folder/test.csv", nrows=500000)

nl()
print('Size of training set: ' + str(df_train.shape))
print(' Size of testing set: ' + str(df_test.shape))

nl()
print('Columns in train: ' + str(df_train.columns.tolist()))
print(' Columns in test: ' + str(df_test.columns.tolist()))

nl()
print(df_train.describe())



Size of training set: (500000, 11)
 Size of testing set: (500000, 7)


Columns in train: ['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID', 'Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima', 'Dev_proxima', 'Demanda_uni_equil']
 Columns in test: ['id', 'Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Cliente_ID', 'Producto_ID']


         Semana     Agencia_ID       Canal_ID       Ruta_SAK    Cliente_ID  \
count  500000.0  500000.000000  500000.000000  500000.000000  5.000000e+05   
mean        3.0    1115.814010       1.118138    1590.864712  1.198001e+06   
std         0.0       3.127955       0.823334     801.433471  1.462295e+06   
min         3.0    1110.000000       1.000000       1.000000  3.967000e+03   
25%         3.0    1112.000000       1.000000    1109.000000  1.098650e+05   
50%         3.0    1117.000000       1.000000    1447.000000  5.137590e+05   
75%         3.0    1118.000000       1.000000    1606.000000  1.793000e+06   
max         3.0   

In [3]:
df_train.head(10)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.14,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.0,3
5,3,1110,7,3301,15766,1250,5,38.20,0,0.0,5
6,3,1110,7,3301,15766,1309,3,20.28,0,0.0,3
7,3,1110,7,3301,15766,3894,6,56.10,0,0.0,6
8,3,1110,7,3301,15766,4085,4,24.60,0,0.0,4
9,3,1110,7,3301,15766,5310,6,31.68,0,0.0,6


In [5]:
categorical_variables = df_train.dtypes.loc[df_train.dtypes == 'object'].index
print(categorical_variables)

Index([], dtype='object')


In [6]:
df_train.apply(lambda x: sum(x.isnull()))

Semana               0
Agencia_ID           0
Canal_ID             0
Ruta_SAK             0
Cliente_ID           0
Producto_ID          0
Venta_uni_hoy        0
Venta_hoy            0
Dev_uni_proxima      0
Dev_proxima          0
Demanda_uni_equil    0
dtype: int64

In [8]:
df_train['Difference']='Train' 
df_test['Difference']='Test'
fullData = pd.concat([df_train, df_test], axis=0)
fullData.Difference.describe()

count     1000000
unique          2
top          Test
freq       500000
Name: Difference, dtype: object

In [9]:
ID_col=['id']
target_col=['Demanda_uni_equil']
cat_col=['Difference']
features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(cat_col))

In [12]:
cv = []
lst = []
kf = cross_validation.KFold(len(df_train), n_folds=5, random_state=0)
for idx1, idx2 in kf:
    x_train, x_cv = df_train[features].iloc[idx1], df_train[features].iloc[idx2]
    y_train, y_cv = df_train.Demanda_uni_equil.iloc[idx1], df_train.Demanda_uni_equil.iloc[idx2]
    
    random.seed(100)
    rf = RandomForestRegressor(n_estimators=500,max_depth=20,max_leaf_nodes=2)
    rf.fit(x_train, y_train)
    cv.extend(rf.predict(x_cv))
    lst.append(rf.predict(df_test[features]))
    print(metrics.mean_squared_error(rf.predict(x_cv), y_cv))

KeyError: "['Dev_uni_proxima' 'Dev_proxima' 'Venta_hoy' 'Venta_uni_hoy'] not in index"